# 2 - Calculate Nino 3.4 time series for ACCESS-1.3

In this exercise you will calculate the Nino 3.4 index for the ACCESS-1.3 CMIP5 submission using the sea surafce temperature from the ocean model component

Definition of the Nino 3.4 index from https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni

> Niño 3.4 (5N-5S, 170W-120W):  The  Niño 3.4 anomalies may be thought of as representing the average equatorial SSTs across the Pacific from about the dateline to the South American coast.  The Niño 3.4 index typically uses a 5-month running mean, and El Niño or La  Niña events are defined when the  Niño 3.4 SSTs exceed +/- 0.4C for a period of six months or more.

> Nino X Index computation: (a) Compute area averaged total SST from Niño X region; (b) Compute monthly climatology (e.g., 1950-1979) for area averaged total SST from Niño X region, and subtract climatology from area averaged total SST time series to obtain anomalies; (c) Smooth the anomalies with a 5-month running mean; (d) Normalize the smoothed values by its standard deviation over the climatological period.

The URL for the relevant dataset is 

http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/ocean/Omon/r1i1p1/latest/tos/tos_Omon_ACCESS1-3_historical_r1i1p1_185001-200512.nc

First step is to open the dataset (remember to import any packages you will need)

Find the sea surface temperature (SST) variable name and create a new variable which selects out all data after 1950. Probably a good idea to plot the first time slice as a reference

MOM5 is the ocean model component of ACCESS-1.3. In this configuration the ocean has a tripolar grid. This means there are three poles, to avoid having a convergent point in the ocean. All three poles are over land points where there are no wet cells. One pole in the south over Antarctica and two in the north. What this means is that the data have coordinates `i` and `j` and the `lat` and `lon` data variables are 2D and also have coordinates `i` and `j`. The tripolar grid only starts diverging from a normal above 60N, so does not affect the calculation of Nino 3.4, but it does make it more difficult to select out the area of interest.

![Tripole Grid](tripole.png)

There are a few ways selecting out the region of interest (ROI). 

1. Find the cell indices for the ROI and use `sel`
2. Chop off the tripole part of the grid and add new 1D lat/lon coordinates to the dataset/variable
3. Use `where` and select for all values within the bounds of interest.

It is the last option that is easiest, for variable `var` it is formulated like so

`var.where((var.lat<5) & (var.lat>-5) & (var.lon>190) & (var.lon<240),drop=True)`

note the `drop=True` option, which will remove all masked cells, which in this case is equivalent to a `sel` operation usin `slice` on `lat` and `lon`

Calculate a monthly climatology of mean SST for the 1950-1979 period. Note this should be a single value for each month

Now calculate the Nino 3.4 index, by subtracting the monthly climatology from the original SST data and take the mean over the spatial coordinates

Next step is to apply a 5 month moving average smoother. xarray has a method [rolling](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.rolling.html) which can be applied to the your saved variable and then directly plotted

Last step is to normalise by the standard deviation. Calculate the standard deviation of the SST data in the Nino 3.4 area for the climatological period. xarray provides the [std](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.std.html?highlight=std) operator for this purpose. Save the result in a new variable. Note that this is an average over the entire climatological period, so should be a single value

As the standard deviation is a single value it is sufficient to use the same formulation for your plot as above, but divide by the standard deviation and then plot the result. Note that parentheses will be required around the divide operation to correctly chain the `plot()` operator. e.g. something like

`(index_variable.rolling().mean() / std_dev).plot()`

Alternatively you could save the normalised value into a new variable and then plot that